# 02 RAG | 01 Chat Completion Text | 01 REST

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Intro

Chat models take a series of messages as input, and return a model-generated message as output. The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).

## Step 1:  Setup Parameters

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/Configuration/) to setup the necessary demo environment.

In [1]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT"); 
string apiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY"); 
string deploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME"); 

Console.WriteLine("Connection information loaded from configuration file...");


Installed Packages DotNetEnv, 2.5.0 Newtonsoft.Json, 13.0.1 System.Text.Json, 7.0.3

Connection information loaded from configuration file...


## Step 2:  Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

System prompt: `"You are an AI assistance who helps finding information around international sport events"`

Prompt: `"Who won the super bowl 2024?"`

In [11]:

var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an AI assistance who helps finding information around international sport events" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "Who won the Super Bowl 2024?" }
            },
        }
    },
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

Console.WriteLine($"Request payload defined ...");

Request payload defined ...


## Step 3:  Call Azure OpenAI REST endpoint

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance. As the deployed GPT-4o model has training data including October 2023 it will not be able to answer this question correctly. This behavior is expected and methods to address this challenge are shown in e.g. [02_ChatCompletionText_SDK](./02_ChatCompletionText_SDK.ipynb).

In [12]:
string apiVersion = "2024-02-01";
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}


Output: As of my last update in October 2023, the Super Bowl LVIII (58) is scheduled to take place on February 11, 2024, at Allegiant Stadium in Paradise, Nevada. The winner of the Super Bowl 2024 has not been determined yet. Please check the latest sources or news updates for the most current information.
